In [46]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import time

spark = (
    SparkSession
    .builder
    .config("spark.sql.warehouse.dir", "./temp")
    .master('local[*]')
    .getOrCreate()
)

In [30]:
df_transacoes_spark = spark.read.csv(r"C:\dados-tcs-itau\Etapa 4\dados\transacoes.csv", header=True, inferSchema=True)
df_transacoes_spark.show()

df_clientes_spark = spark.read.csv(r"C:\dados-tcs-itau\Etapa 4\dados\clientes.csv", header=True, inferSchema=True)
df_clientes_spark.show()

+------------+----------+-----+--------------+-----------+
|id_transacao|id_cliente|valor|data_transacao|  categoria|
+------------+----------+-----+--------------+-----------+
|           1|         1|100.0|    2023-11-01|    Compras|
|           2|         2|200.0|    2023-11-02|Alimentação|
|           3|         3|150.0|    2023-11-03|    Compras|
|           4|         4|300.0|    2023-11-01|     Viagem|
|           5|         5| 50.0|    2023-11-04|Alimentação|
+------------+----------+-----+--------------+-----------+

+----------+-----+-----+--------------+
|id_cliente| nome|idade|        cidade|
+----------+-----+-----+--------------+
|         1| João|   35|     São Paulo|
|         2|Maria|   25|Rio de Janeiro|
|         3| José|   40|      Curitiba|
|         4|  Ana|   28|     São Paulo|
|         5|Paulo|   50|Belo Horizonte|
+----------+-----+-----+--------------+



In [53]:
df_transacoes_pandas = pd.read_csv(r"C:\dados-tcs-itau\Etapa 4\dados\transacoes.csv")
print(df_transacoes_pandas)
print()
df_clientes_pandas = pd.read_csv(r"C:\dados-tcs-itau\Etapa 4\dados\clientes.csv")
print(df_clientes_pandas)

   id_transacao  id_cliente  valor data_transacao    categoria
0             1           1  100.0     2023-11-01      Compras
1             2           2  200.0     2023-11-02  Alimentação
2             3           3  150.0     2023-11-03      Compras
3             4           4  300.0     2023-11-01       Viagem
4             5           5   50.0     2023-11-04  Alimentação

   id_cliente   nome  idade          cidade
0           1   João     35       São Paulo
1           2  Maria     25  Rio de Janeiro
2           3   José     40        Curitiba
3           4    Ana     28       São Paulo
4           5  Paulo     50  Belo Horizonte


In [52]:
start_time = time.time()
df_clientes_spark.filter(df_clientes_spark["idade"] > 30).show()
end_time = time.time()
print(f"Tempo de execução com PySpark: {end_time - start_time:.6f} segundos")

+----------+-----+-----+--------------+
|id_cliente| nome|idade|        cidade|
+----------+-----+-----+--------------+
|         1| João|   35|     São Paulo|
|         3| José|   40|      Curitiba|
|         5|Paulo|   50|Belo Horizonte|
+----------+-----+-----+--------------+

Tempo de execução com PySpark: 0.092539 segundos


In [35]:
df = pd.DataFrame(df_clientes_pandas)
clientes_30 = df[df["idade"] > 30]
print(clientes_30)

   id_cliente   nome  idade          cidade
0           1   João     35       São Paulo
2           3   José     40        Curitiba
4           5  Paulo     50  Belo Horizonte


In [51]:
start_time = time.time()

df_resultado = df_transacoes_spark.groupBy("categoria") \
    .agg(
        F.sum("valor").alias("valor_total"),
        F.avg("valor").alias("valor_medio")
    ) \
    .orderBy(F.col("valor_total").desc())
    
df_resultado.show()
end_time = time.time()
print(f"Tempo de execução com PySpark: {end_time - start_time:.6f} segundos")

+-----------+-----------+-----------+
|  categoria|valor_total|valor_medio|
+-----------+-----------+-----------+
|     Viagem|      300.0|      300.0|
|    Compras|      250.0|      125.0|
|Alimentação|      250.0|      125.0|
+-----------+-----------+-----------+

Tempo de execução com PySpark: 0.216827 segundos


In [50]:
start_time = time.time()
df = pd.DataFrame(df_transacoes_pandas)

df_resultado = df.groupby("categoria").agg(
    valor_total=("valor", "sum"),
    valor_medio=("valor", "mean")
)

print(df_resultado)
end_time = time.time()
print(f"Tempo de execução com Pandas: {end_time - start_time:.6f} segundos")

             valor_total  valor_medio
categoria                            
Alimentação        250.0        125.0
Compras            250.0        125.0
Viagem             300.0        300.0
Tempo de execução com Pandas: 0.007562 segundos
